# Exploring Anthracyclines and Disease within an OMOP CDM Database

**Date:** June 22 2024

**Summary:** An overview of how to use the observational health tools within JuliaHealth to explore the usage of various Anthracyclines alongside diseases of interest.

**Keywords:** #cvd #disease #observational #health #anthracycline

## Bibliography

Not Available

## Notes

### Motivation

This is a small sample notebook I threw together to illustrate how to further build a small characterization study when exploring drugs and conditions of interest.
It goes into a bit more depth on how to find drugs and diseases folks are interested in.

### Set-Up

Activating the environment (note: this requires `DrWatson.jl` to be installed in your default Julia environment):

In [1]:
using DrWatson
@quickactivate "CVDAnthracyclines"

Loading required packages

In [2]:
using JSON3
using HTTP
using OMOPCDMCohortCreator

import DBInterface:
    connect,
    execute
import LibPQ:
    Connection

Creating connection to database

In [3]:
conn = connect(
    Connection, 
    "user=thecedarprince dbname=synthea"
)

DETAIL:  The database was created using collation version 2.37, but the operating system provides version 2.38.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE synthea REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


PostgreSQL connection (CONNECTION_OK) with parameters:
  user = thecedarprince
  passfile = /home/thecedarprince/.pgpass
  channel_binding = prefer
  dbname = synthea
  port = 5432
  client_encoding = UTF8
  options = -c DateStyle=ISO,YMD -c IntervalStyle=iso_8601 -c TimeZone=UTC
  application_name = LibPQ.jl
  sslmode = prefer
  sslcompression = 0
  sslsni = 1
  ssl_min_protocol_version = TLSv1.2
  gssencmode = prefer
  krbsrvname = postgres
  target_session_attrs = any

Generating database details for the synthetic database we are connecting to (this is used internally by the `OMOPCDMCohortCreator.jl` packag):

In [4]:
GenerateDatabaseDetails(:postgresql, "omop")
GenerateTables(conn)

[ Info: Global database dialect set to: postgresql
[ Info: Global schema set to: omop


[ Info: source_to_concept_map table generated internally
[ Info: payer_plan_period table generated internally
[ Info: vocabulary table generated internally
[ Info: note_nlp table generated internally
[ Info: procedure_occurrence table generated internally
[ Info: domain table generated internally
[ Info: metadata table generated internally
[ Info: cohort_summary_stats table generated internally
[ Info: fact_relationship table generated internally
[ Info: drug_exposure table generated internally
[ Info: all_visits table generated internally
[ Info: final_visit_ids table generated internally
[ Info: person table generated internally
[ Info: cost table generated internally
[ Info: observation table generated internally
[ Info: provider table generated internally
[ Info: visit_detail table generated internally
[ Info: assign_all_visit_ids table generated internally
[ Info: device_exposure table generated internally
[ Info: measurement table generated internally
[ Info: source_to_standard_v

PostgreSQL connection (CONNECTION_OK) with parameters:
  user = thecedarprince
  passfile = /home/thecedarprince/.pgpass
  channel_binding = prefer
  dbname = synthea
  port = 5432
  client_encoding = UTF8
  options = -c DateStyle=ISO,YMD -c IntervalStyle=iso_8601 -c TimeZone=UTC
  application_name = LibPQ.jl
  sslmode = prefer
  sslcompression = 0
  sslsni = 1
  ssl_min_protocol_version = TLSv1.2
  gssencmode = prefer
  krbsrvname = postgres
  target_session_attrs = any

### Creating Drug and Disease Look-Ups

Here are the anthracyclines we are exploring:

In [5]:
drugs = [
    (name = "Daunorubicin", id = 1311799),
    (name = "Doxorubicin", id = 1338512),
    (name = "Epirubicin", id = 1344354),
    (name = "Idarubicin", id = 19078097),
    (name = "Mitoxantrone", id = 1309188),
    (name = "Valrubicin", id = 19012543),
    (name = "Ibrutinib", id = 44507848),
    (name = "Acalabrutinib", id = 792764),
    (name = "Zanubrutinib", id = 37497691),
]

9-element Vector{@NamedTuple{name::String, id::Int64}}:
 (name = "Daunorubicin", id = 1311799)
 (name = "Doxorubicin", id = 1338512)
 (name = "Epirubicin", id = 1344354)
 (name = "Idarubicin", id = 19078097)
 (name = "Mitoxantrone", id = 1309188)
 (name = "Valrubicin", id = 19012543)
 (name = "Ibrutinib", id = 44507848)
 (name = "Acalabrutinib", id = 792764)
 (name = "Zanubrutinib", id = 37497691)

Here are the diseases we are exploring:

In [6]:
diseases = [
    (name = "Myocardial Infarction", id = 4329847),
    (name = "Heart Failure", id = 316139),
]

2-element Vector{@NamedTuple{name::String, id::Int64}}:
 (name = "Myocardial Infarction", id = 4329847)
 (name = "Heart Failure", id = 316139)

For both these drugs and diseases, the IDs come from [ATLAS](https://atlas-demo.ohdsi.org/#/search), a tool to develop computable phenotype definitions (i.e. translating a disease phenotype definition to the same thing but in code).
How these IDs were found for a drug is as follows (using Doxorubicin as an example):

1. Search within ATLAS for Doxorubicin.

2. Click any Doxorubicin-related drug name.

3. Look through the related concepts or hierarchy (in the parent section) for the ingredient version of the drug.

4. Copy that ID for the ingredient version of Doxorubicin.

That is the ID we used here as it is the base for all other Doxorubicin medications.

Next, we can do something similar for Myocardial Infarction:

1. Search within ATLAS for Myocardial Infarction.

2. Click any Myocardial Infarction-related name.

3. Look through the hierarchy of this disease. 
The condition version we are looking for is that parent concepts are too broad for our condition and the current condition we select should have children that encompass all diseases we want (in this case, this is [recorded condition](https://atlas-demo.ohdsi.org/#/concept/4329847) is the perfect level of granularity).

4. Copy that ID for the ingredient version of Myocardial Infarction.

That is the ID we used here as it is the base for all other Myocardial Infarction-related disease conditions.

Next, we'll discuss how to create a query that includes the derivatives of the general concepts for the drugs and diseases we found.

### Generating Ad Hoc Definitions

Here, we programmatically query the public ATLAS instance that is running to get the children IDs of the general concepts (referred to as descendants in ATLAS).
In this case, we are, in a very ad hoc way, creating an implicit computable phenotype definition where we are looking for all drugs and diseases that are based off our general concepts.
This is great for rapid prototyping, but a phenotype definition must be scrutinized ruthlessly when moving towards a final version for patient analysis (see: [Zelko, Jacob S., et al. "Developing a Robust Computable Phenotype Definition Workflow to Describe Health and Disease in Observational Health Research." arXiv preprint arXiv:2304.06504 (2023).](https://arxiv.org/abs/2304.06504) which discussed more about this process).

This is how we do that for drugs:

In [7]:
drug_descendants = []
for drug in drugs
    path = "https://atlas-demo.ohdsi.org/WebAPI/vocabulary/ATLASPROD/concept/$(drug.id)/descendants/"
    descendants = 
        HTTP.get(path) |> 
        x -> String(x.body) |> 
        JSON3.read
    push!(drug_descendants, descendants)
end

This is how we do that for diseases:

In [8]:
disease_descendants = []
for disease in diseases
    path = "https://atlas-demo.ohdsi.org/WebAPI/vocabulary/ATLASPROD/concept/$(disease.id)/descendants/"
    descendants = 
        HTTP.get(path) |> 
        x -> String(x.body) |> 
        JSON3.read
    push!(disease_descendants, descendants)
end

### Analyzing the OMOPCDM Database

Find users of the drugs we are interested in:

In [9]:
drug_users = Dict()
for (idx, drug) in enumerate(drugs)
    ids = OMOPCDMCohortCreator.DrugExposureFilterPersonIDs(
        [
            drug[:CONCEPT_ID] 
            for drug in drug_descendants[idx]
        ], conn).person_id
    push!(
        drug_users, 
        drug.name => 
            (ids = ids, count = length(ids))
    )
end

DETAIL:  The database was created using collation version 2.37, but the operating system provides version 2.38.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE synthea REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.37, but the operating system provides version 2.38.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE synthea REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.37, but the operating system provides version 2.38.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE synthea REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.37, but the operating system provides version 2.3

Find patients diagnosed with the conditions we are interested in:

In [10]:
diseased_patients = Dict()
for (idx, disease) in enumerate(diseases)
    ids = ConditionFilterPersonIDs(
        [
            d[:CONCEPT_ID] 
            for d in disease_descendants[idx]
            ], conn).person_id
    push!(
        diseased_patients, 
        disease.name => 
            (ids = ids, count = length(ids))
        )
end

DETAIL:  The database was created using collation version 2.37, but the operating system provides version 2.38.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE synthea REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.37, but the operating system provides version 2.38.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE synthea REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.37, but the operating system provides version 2.38.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE synthea REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.37, but the operating system provides version 2.3

Find the combination of patients diagnosed with the conditions we are interested in and taking the drugs we want to investigate:

In [11]:
condition_mix = Dict()
for pg in keys(diseased_patients)
    push!(condition_mix, pg => Dict())
    for du in keys(drug_users)
        mix = intersect(
            diseased_patients[pg].ids, 
            drug_users[du].ids
        )
        push!(
            condition_mix[pg], 
            du => (ids = mix, count = length(mix))
        )
    end
end

### Saving Results

In [12]:
open(datadir("exp_raw", "drug_user_counts.csv"), "w") do io
    write(io, "DRUG,COUNT\n")
    for du in keys(drug_users)
        write(io, "$du,$(drug_users[du].count)\n")
    end
end

In [13]:
open(datadir("exp_raw", "diseased_patient_counts.csv"), "w") do io
    write(io, "DISEASE,COUNT\n")
    for pg in keys(diseased_patients)
        write(io, "$pg,$(diseased_patients[pg].count)\n")
    end
end

In [14]:
open(datadir("exp_raw", "mix_counts.csv"), "w") do io
    write(io, "DISEASE,DRUG,COUNT\n")
    for disease in keys(condition_mix)
        for drug in keys(condition_mix[disease])
            write(
                io, 
                """
                $disease,
                $drug,
                $(condition_mix[disease][drug].count)\n
                """
            )
        end
    end
end

## References: